In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
import contextily as cx
import rtree

/Users/jnapolitano/venvs/sphinx-build/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
## Importing our DataFrames

gisfilepath = "/Users/jnapolitano/Projects/rail-mapping/intermodal/Intermodal_Freight_Facilities_RailTOFCCOFC.geojson"

rail_to_all_df = gpd.read_file(gisfilepath)

rail_to_all_df = rail_to_all_df.to_crs(epsg=3857)

rail_to_all_df

,OBJECTID,TERMINAL,TERM_ADDRE,CITY,STATE,ZIP_CODE,PORT,DATE,RAIL_COMPA,EQUIPMENT,ACCESS_ROA,SPLC,LAT,LON,geometry
0,1,"Albany, NY - Mechanicville",50 Route 67,Mechanicville,NY,12118,,2018-08-01T00:00:00+00:00,NS,COFC,,171760,42.914715,-73.709533,POINT (-8205307.680 5298999.597)
1,2,"Albuquerque, NM",100 Woodward Road S.E.,Albuquerque,NM,87102,,2018-09-01T00:00:00+00:00,BNSF,TOFC/COFC,,774900,35.047580,-106.653344,POINT (-11872595.946 4170348.957)
2,3,"Alliance, TX",1111 Intermodal Parkway,Haslet,TX,76052,,2018-09-02T00:00:00+00:00,BNSF,TOFC/COFC,,668100,32.999881,-97.347026,POINT (-10836621.365 3895288.168)
3,4,"Arcadia, WI",One Ashley Way,Arcadia,WI,54612,,2018-09-01T00:00:00+00:00,CN,TOFC/COFC,South Washington St.,328754,44.251243,-91.509663,POINT (-10186809.088 5504405.295)
4,5,"Atlanta, GA - Austell","6000 Dr. Luke Glenn Garrett, Jr. Memorial Highway",Austell,GA,30106,,2018-08-01T00:00:00+00:00,NS,TOFC/COFC,,456496,33.825798,-84.655069,POINT (-9423759.174 4005434.870)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,177,"Wilmington, CA - TraPac Intermodal Container T...",630 West Harry Bridges Boulevard,Wilmington,CA,90744,Port of Los Angeles,2018-12-07T00:00:00+00:00,PHL,TOFC/COFC,,883257,33.770320,-118.273289,POINT (-13166122.306 3998003.151)
177,178,"Wilmington, NC - Container Terminal",1 Shipyard Boulevard,Wilmington,NC,28401,Port of Wilmington,2018-09-11T00:00:00+00:00,CSX,COFC,,409160,34.191224,-77.952419,POINT (-8677623.589 4054507.730)
178,179,"Winter Haven, FL - Central Florida Intermodal ...",3935 Intermodal Drive,Winter Haven,FL,33884,,2018-09-11T00:00:00+00:00,"CSX, EVWR",COFC,,497540,27.939953,-81.693226,POINT (-9094048.320 3241405.344)
179,180,"Worcester, MA",165 Grafton Street,Worcester,MA,01604,,2018-09-11T00:00:00+00:00,CSX,TOFC/COFC,,145460,42.259951,-71.789244,POINT (-7991542.086 5199998.731)


In [3]:
gisfilepath = "/Users/jnapolitano/Projects/rail-mapping/shipping/Major_Ports.geojson"
major_ports_df = gpd.read_file(gisfilepath)
major_ports_df = major_ports_df.to_crs(epsg=3857)

major_ports_df

,OBJECTID_1,OBJECTID,ID,PORT,PORT_NAME,GRAND_TOTA,FOREIGN_TO,IMPORTS,EXPORTS,DOMESTIC,geometry
0,1,1,124,C4947,"Unalaska Island, AK",1652281,1236829,426251,810578,415452,POINT (-18806114.399 7124760.122)
1,2,2,85,C4410,"Kahului, Maui, HI",3615449,20391,20391,0,3595058,POINT (-17418472.419 2379793.012)
2,3,3,116,C4405,"Kawaihae Harbor, HI",2034621,0,0,0,2034621,POINT (-17347283.605 2277507.024)
3,4,4,113,C4430,"Nawiliwili, Kauai, HI",2069212,22066,22066,0,2047146,POINT (-17739105.948 2506856.666)
4,5,5,52,C4458,"Barbers Point, Oahu, HI",9854176,7384757,6683744,701013,2469419,POINT (-17600602.238 2427355.584)
...,...,...,...,...,...,...,...,...,...,...,...
145,146,146,145,C4240,"Sacramento, CA",898044,898044,733008,165036,0,POINT (-13625399.029 4552149.948)
146,147,147,142,I2094,"Morgan City, LA, Port of",991600,51,51,0,991549,POINT (-10161297.777 3429249.067)
147,148,148,144,I6111,"Little Rock, AR, Port of",914190,0,0,0,914190,POINT (-10281276.811 4135486.697)
148,149,149,150,I6112,"Fort Smith, AR, Port of",788939,0,0,0,788939,POINT (-10506939.342 4224548.117)


In [4]:
major_transit_nodes = rail_to_all_df.sjoin_nearest(major_ports_df)
major_transit_nodes.explore()

In [5]:
gisfilepath = "/Users/jnapolitano/Projects/rail-mapping/intermodal/Intermodal_Freight_Facilities_Air-to-Truck.geojson"
air_freight_to_truck_df = gpd.read_file(gisfilepath)
air_freight_to_truck_df = air_freight_to_truck_df.to_crs(epsg=3857)

air_freight_to_truck_df

,OBJECTID,LOCID,FACILITY_C,DATE,EST_AREA,geometry
0,1,PIT,Delta Air Lines; United Parcel Service,09/2018,74000,POINT (-8932265.943 4939618.132)
1,2,PIT,Federal Express,09/2018,73000,POINT (-8931996.093 4939519.257)
2,3,PIT,,09/2018,65000,POINT (-8931632.274 4939487.129)
3,4,PIT,,09/2018,55000,POINT (-8930019.150 4938284.560)
4,5,RDU,Federal Express,09/2018,63000,POINT (-8769327.059 4285470.308)
...,...,...,...,...,...,...
399,400,IAH,British Airways; Swissport Cargo Services,12/2018,88000,POINT (-10609992.124 3503138.495)
400,401,IAH,Air General; American Airlines; Dachser,12/2018,70000,POINT (-10609855.207 3503137.177)
401,402,IAH,Cargo Airport Services Texas; Cargolux Airline...,12/2018,149000,POINT (-10609937.414 3502943.849)
402,403,IAH,Singapore Airlines,12/2018,149000,POINT (-10609922.972 3502538.518)


In [6]:
major_air_freight = air_freight_to_truck_df.sjoin_nearest(major_ports_df)
major_air_freight

,OBJECTID_left,LOCID,FACILITY_C,DATE,EST_AREA,geometry,index_right,OBJECTID_1,OBJECTID_right,ID,PORT,PORT_NAME,GRAND_TOTA,FOREIGN_TO,IMPORTS,EXPORTS,DOMESTIC
0,1,PIT,Delta Air Lines; United Parcel Service,09/2018,74000,POINT (-8932265.943 4939618.132),113,114,114,33,I2358,"Pittsburgh, PA",21567015,0,0,0,21567015
1,2,PIT,Federal Express,09/2018,73000,POINT (-8931996.093 4939519.257),113,114,114,33,I2358,"Pittsburgh, PA",21567015,0,0,0,21567015
2,3,PIT,,09/2018,65000,POINT (-8931632.274 4939487.129),113,114,114,33,I2358,"Pittsburgh, PA",21567015,0,0,0,21567015
3,4,PIT,,09/2018,55000,POINT (-8930019.150 4938284.560),113,114,114,33,I2358,"Pittsburgh, PA",21567015,0,0,0,21567015
4,5,RDU,Federal Express,09/2018,63000,POINT (-8769327.059 4285470.308),114,115,115,70,C766,"Wilmington, NC",6039927,5660231,3377654,2282577,379696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,389,HNL,Alaska Airlines,12/2018,42000,POINT (-17578969.072 2429783.328),5,6,6,41,C4420,"Honolulu, HI",15181890,1439915,1146253,293662,13741975
389,390,HNL,United Parcel Service,12/2018,30000,POINT (-17578874.086 2429878.269),5,6,6,41,C4420,"Honolulu, HI",15181890,1439915,1146253,293662,13741975
390,391,HNL,American Airlines; Delta Air Lines,12/2018,20000,POINT (-17578581.567 2431567.128),5,6,6,41,C4420,"Honolulu, HI",15181890,1439915,1146253,293662,13741975
391,392,HSV,,12/2018,130000,POINT (-9658266.854 4115396.770),82,83,83,119,I2371,"Guntersville, AL",1918572,0,0,0,1918572


In [7]:
major_air_freight.explore()